# Review DPS Outputs

Make a mosaic of DPS outputs.

1. make a list of the DPS output paths
2. merge that list with the original index tile file
3. convert the local paths to s3 paths
4. reproject the bounds to 4326 - convert to geojson (save file or in mem object)
5. make mosaicjson Mosaic.from_features
6. send mosaicjson to the tile server

In [8]:
import geopandas as gpd
import pandas as pd
import os
import json

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning:

The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.



## Build a Geojson with all the data

In [11]:
def local_to_s3(url):
    ''' A Function to convert local paths to s3 urls'''
    user = 'nathanmthomas'
    return url.replace('/projects/my-private-bucket', f's3://maap-ops-workspace/{user}')

In [12]:
# We just need a list of files
# TODO: skip making a shapefile from the outputs
#outputs = gpd.read_file("/projects/DPS/Landsat_tindex_350.shp")
outputs = pd.read_csv("/projects/DPS/Landsat_tindex_master.csv")
output_local = outputs['location'].tolist()
output_local[0:10]

['/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/08/18/22/31/53/993956/Landsat8_1602_comp_cog_2015-2020_dps.tif',
 '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/08/18/22/33/08/154474/Landsat8_1042_comp_cog_2015-2020_dps.tif',
 '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/08/18/22/33/48/946129/Landsat8_1097_comp_cog_2015-2020_dps.tif',
 '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/08/18/22/35/17/849806/Landsat8_1038_comp_cog_2015-2020_dps.tif',
 '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/08/18/22/35/19/285754/Landsat8_1151_comp_cog_2015-2020_dps.tif',
 '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/08/18/22/35/30/330959/Landsat8_1040_comp_cog_2015-2020_dps.tif',
 '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/08/18/22/35/35/909196/Landsat8_1204_comp_cog_2015-

In [13]:
# Build up a dataframe from the list of dps output files
base_frame = pd.DataFrame(output_local, columns=['local_path'])
base_frame['id'] = [int(os.path.basename(local_path).split('_')[1]) for local_path in base_frame['local_path']]
base_frame['s3'] = [local_to_s3(local_path) for local_path in base_frame['local_path']]

In [14]:
base_frame.sort_values('id', inplace=True)
#base_frame.set_index('id', drop=True, inplace=True)
base_frame.head()


,local_path,id,s3
317,/projects/my-private-bucket/dps_output/do_land...,358,s3://maap-ops-workspace/nathanmthomas/dps_outp...
335,/projects/my-private-bucket/dps_output/do_land...,389,s3://maap-ops-workspace/nathanmthomas/dps_outp...
329,/projects/my-private-bucket/dps_output/do_land...,420,s3://maap-ops-workspace/nathanmthomas/dps_outp...
367,/projects/my-private-bucket/dps_output/do_land...,421,s3://maap-ops-workspace/nathanmthomas/dps_outp...
310,/projects/my-private-bucket/dps_output/do_land...,451,s3://maap-ops-workspace/nathanmthomas/dps_outp...


In [9]:
vector_path = '/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg'
tile_index = gpd.read_file(vector_path)
tile_index.astype({'layer':'int'})
tile_index.rename(columns={"layer":"id"}, inplace=True)
tile_index.head()

,id,geometry
0,1.0,"POLYGON ((-2151478.000 9423304.000, -2061478.0..."
1,2.0,"POLYGON ((-2061478.000 9423304.000, -1971478.0..."
2,3.0,"POLYGON ((-1971478.000 9423304.000, -1881478.0..."
3,4.0,"POLYGON ((-2241478.000 9333304.000, -2151478.0..."
4,5.0,"POLYGON ((-2151478.000 9333304.000, -2061478.0..."


In [15]:
# Select the rows we have results for
tile_matches = tile_index.iloc[base_frame['id'].tolist()].merge(base_frame, on='id')

In [16]:
tile_matches.head()

,id,geometry,local_path,s3
0,421.0,"POLYGON ((4508522.000 7443304.000, 4598522.000...",/projects/my-private-bucket/dps_output/do_land...,s3://maap-ops-workspace/nathanmthomas/dps_outp...
1,452.0,"POLYGON ((4238522.000 7353304.000, 4328522.000...",/projects/my-private-bucket/dps_output/do_land...,s3://maap-ops-workspace/nathanmthomas/dps_outp...
2,453.0,"POLYGON ((4328522.000 7353304.000, 4418522.000...",/projects/my-private-bucket/dps_output/do_land...,s3://maap-ops-workspace/nathanmthomas/dps_outp...
3,454.0,"POLYGON ((4418522.000 7353304.000, 4508522.000...",/projects/my-private-bucket/dps_output/do_land...,s3://maap-ops-workspace/nathanmthomas/dps_outp...
4,455.0,"POLYGON ((4508522.000 7353304.000, 4598522.000...",/projects/my-private-bucket/dps_output/do_land...,s3://maap-ops-workspace/nathanmthomas/dps_outp...


In [17]:
geojson_string = tile_matches.to_crs("EPSG:4326").to_json()
geojson = json.loads(geojson_string)

In [18]:
geojson.get('features')[10]

{'id': '10',
 'type': 'Feature',
 'properties': {'id': 495.0,
  'local_path': '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/08/19/04/14/28/610291/Landsat8_495_comp_cog_2015-2020_dps.tif',
  's3': 's3://maap-ops-workspace/nathanmthomas/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/08/19/04/14/28/610291/Landsat8_495_comp_cog_2015-2020_dps.tif'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-54.92051175664547, 50.98372233224968],
    [-55.28264345232914, 50.2091605991493],
    [-56.49918632083444, 50.43006548329133],
    [-56.15761461645353, 51.20827592505761],
    [-54.92051175664547, 50.98372233224968]]]}}

## Build a MosaicJSON

In [104]:
from typing import Dict

from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

In [105]:
def get_accessor(feature: Dict):
    """Return specific feature identifier."""
    return feature["properties"]["s3"]

In [106]:
mosaicdata = MosaicJSON.from_features(geojson.get('features'), minzoom=8, maxzoom=16, accessor=get_accessor)

In [107]:
mosaic_json = "/projects/my-public-bucket/Landsat/landsat8_mosaic.json"
with MosaicBackend(mosaic_json, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)
    
# URL to give tiler is
# s3://maap-ops-workspace/shared/nathanmthomas/Landsat/landsat8_mosaic.json

## View the Results

In [1]:
import ipycmc
import urllib

In [2]:
from folium import Map, TileLayer, GeoJson, LayerControl

In [3]:
wmts_url = "https://jqsd6bqdsf.execute-api.us-west-2.amazonaws.com/mosaicjson/WMTSCapabilities.xml"
params = {
    "tile_format":"png",
    "tile_scale":"1",
    "pixel_selection":"first",
    "TileMatrixSetId":"WebMercatorQuad",
    "url":"s3://maap-ops-workspace/shared/nathanmthomas/Landsat/landsat8_mosaic.json",
    "bidx":"6", # Select which band to use
    "resampling_method":"nearest",
    "rescale":"0,1", # Values in data are from 0 to 1
    "return_mask":"true",
    "colormap_name":"magma" # Any colormap from matplotlib will work
}

wmts_call = "?".join([wmts_url, urllib.parse.urlencode(params)])

# Note Jupyter bug add amp; incorrectly when printing the url
wmts_call

'https://jqsd6bqdsf.execute-api.us-west-2.amazonaws.com/mosaicjson/WMTSCapabilities.xml?tile_format=png&tile_scale=1&pixel_selection=first&TileMatrixSetId=WebMercatorQuad&url=s3%3A%2F%2Fmaap-ops-workspace%2Fshared%2Fnathanmthomas%2FLandsat%2Flandsat8_mosaic.json&bidx=6&resampling_method=nearest&rescale=0%2C1&return_mask=true&colormap_name=magma'

In [4]:
w = ipycmc.MapCMC()
w

MapCMC()

In [5]:
# This adds a new layer to the map above, call Cloud Optimized GeoTIFF
w.load_layer_config(wmts_call, "wmts/xml")

In [20]:
tiler_base = "https://jqsd6bqdsf.execute-api.us-west-2.amazonaws.com/"
tiler_mosaic =  "".join([tiler_base, "mosaicjson/tiles/{z}/{x}/{y}"])

zoom_start = 8

m1 = Map(
    tiles="OpenStreetMap",
    zoom_start=zoom_start
)

bbox_style = {'fillColor': '#ff0000', 'color': '#ff0000'}
all_tiles = GeoJson(
    data=tile_index.to_crs("EPSG:4326").to_json(),
    style_function=lambda x:bbox_style,
    name="all tiles"
).add_to(m1)

geo_json = GeoJson(
    data=geojson,
    name="dps tiles"
).add_to(m1)

mj_tile = f"{tiler_mosaic}?url=s3://maap-ops-workspace/shared/nathanmthomas/Landsat/landsat8_mosaic.json&rescale=0,1&bidx=6&colormap_name=magma"
tiles = TileLayer(
    tiles=mj_tile,
    opacity=1,
    name="outputs",
    attr="MAAP"
)

tiles.add_to(m1)
LayerControl().add_to(m1)
m1

## Review Single Output

In [114]:
tiler_base = "https://jqsd6bqdsf.execute-api.us-west-2.amazonaws.com/"
tiler_endpoint =  "".join([tiler_base, "cog/tiles/{z}/{x}/{y}"])

In [115]:
item = geojson.get('features')[20]
item

{'id': '20',
 'type': 'Feature',
 'properties': {'id': 571.0,
  'local_path': '/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/08/19/04/12/03/375148/Landsat8_571_comp_cog_2015-2020_dps.tif',
  's3': 's3://maap-ops-workspace/nathanmthomas/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/08/19/04/12/03/375148/Landsat8_571_comp_cog_2015-2020_dps.tif'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-55.99694117104226, 54.52132962689671],
    [-56.37049512874005, 53.74908203051289],
    [-57.70278986227564, 53.955082959461386],
    [-57.354212750393415, 54.7311960121486],
    [-55.99694117104226, 54.52132962689671]]]}}

In [116]:
start = item.get('geometry').get('coordinates')[0][0]
start

[-55.99694117104226, 54.52132962689671]

In [117]:
zoom_start = 8

m = Map(
    tiles="OpenStreetMap",
    location=(start[1], start[0]),
    zoom_start=zoom_start
)

geo_json = GeoJson(
    data=item,
)

geo_json.add_to(m)

cog_tile = f"{tiler_endpoint}?url={item.get('properties').get('s3')}&rescale=0,1&bidx=6"
tiles = TileLayer(
    tiles=cog_tile,
    opacity=1,
    attr="MAAP"
)

tiles.add_to(m)
m

In [44]:
cog_tile

'https://jqsd6bqdsf.execute-api.us-west-2.amazonaws.com//tiles/{z}/{x}/{y}?url=s3://maap-ops-workspace/nathanmthomas/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/08/19/04/09/51/699291/Landsat8_532_comp_cog_2015-2020_dps.tif&rescale=0,1'